In [21]:
# geospatial
from osgeo import gdal, ogr, osr
import rasterio as rst

# base
import numpy as np
import os

In [6]:
# folder locations
scratch = os.path.join("E:\\", "scratch")
f_mask_locs = os.path.join("E:\\", "Merged", "forest_masks")
vlce_locs = os.path.join("E:\\", "Merged", "vlce")

In [7]:
vlces = os.listdir(vlce_locs)
vlces = [os.path.join(vlce_locs, vlce) for vlce in vlces]

In [25]:
for vlce in vlces:
    year = vlce.split("-")[-1][:-4]
    filename = "forest_mask-" + str(year) + ".tif"
    save_loc = os.path.join(f_mask_locs, filename)
    print(save_loc)

    driver = gdal.GetDriverByName('GTiff')
    file = gdal.Open(vlce)
    band = file.GetRasterBand(1)
    lista = band.ReadAsArray()
    
    if not os.path.isfile(save_loc):
        # reclassification; forest classes (81, 210, 220, 230) are valid (1), else invalid (0)
        #lista[np.where((lista != 81) & (lista < 200))] = 0
        #lista[np.where((lista > 200) | (lista == 81))] = 1
        
        lista[np.where(lista == 81)] = 1000
        lista[np.where(lista > 201)] = 1000
        lista[np.where(lista < 201)] = 0
        lista[np.where(lista == 1000)] = 1

        # create new file
        f_mask = driver.Create(save_loc, file.RasterXSize, file.RasterYSize , 1)
        f_mask.GetRasterBand(1).WriteArray(lista)

        # spatial ref system
        proj = file.GetProjection()
        georef = file.GetGeoTransform()
        f_mask.SetProjection(proj)
        f_mask.SetGeoTransform(georef)
        f_mask.FlushCache()

        # close files after being done with them because we aren't animals
        # and we kinda need the memory
        file = None
        band = None
        lista = None
        f_mask = None

H:\Merged\forest_masks\forest_mask-1984.tif
H:\Merged\forest_masks\forest_mask-1985.tif
H:\Merged\forest_masks\forest_mask-1986.tif
H:\Merged\forest_masks\forest_mask-1987.tif
H:\Merged\forest_masks\forest_mask-1988.tif
H:\Merged\forest_masks\forest_mask-1989.tif
H:\Merged\forest_masks\forest_mask-1990.tif
H:\Merged\forest_masks\forest_mask-1991.tif
H:\Merged\forest_masks\forest_mask-1992.tif
H:\Merged\forest_masks\forest_mask-1993.tif
H:\Merged\forest_masks\forest_mask-1994.tif
H:\Merged\forest_masks\forest_mask-1995.tif
H:\Merged\forest_masks\forest_mask-1996.tif
H:\Merged\forest_masks\forest_mask-1997.tif
H:\Merged\forest_masks\forest_mask-1998.tif
H:\Merged\forest_masks\forest_mask-1999.tif
H:\Merged\forest_masks\forest_mask-2000.tif
H:\Merged\forest_masks\forest_mask-2001.tif
H:\Merged\forest_masks\forest_mask-2002.tif
H:\Merged\forest_masks\forest_mask-2003.tif
H:\Merged\forest_masks\forest_mask-2004.tif
H:\Merged\forest_masks\forest_mask-2005.tif
H:\Merged\forest_masks\forest_ma

In [3]:
# to create harvest mask
# no pixels counted that were harvested recently (last ~35 years)
h_mask_loc = os.path.join("E:\\", "Merged", "harvest_mask", "harvest_mask.tif")
if not os.path.isfile(h_mask_loc):
    change = os.path.join("E:\\", "Merged", "Change_Attribution", "BC_Change_Attribution.tif")

    # reclassify change attribution into a harvest mask

    driver = gdal.GetDriverByName('GTiff')
    file = gdal.Open(change)
    band = file.GetRasterBand(1)
    lista = band.ReadAsArray()

    # reclassification; harvest (2) is invalid, everything else is valid
    lista[np.where(lista != 2)] = 1
    lista[np.where(lista == 2) ] = 0

    # create new file
    h_mask = driver.Create(h_mask_loc, file.RasterXSize , file.RasterYSize , 1)
    h_mask.GetRasterBand(1).WriteArray(lista)

    # spatial ref system
    proj = file.GetProjection()
    georef = file.GetGeoTransform()
    h_mask.SetProjection(proj)
    h_mask.SetGeoTransform(georef)
    h_mask.FlushCache()

    # close files after being done with them because we aren't animals
    # and we kinda need the memory
    h_mask = None
    file = None
    lista = None